In [ ]:
import os
os.environ['PYNQ_REMOTE_IP'] = "192.168.0.114"

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pynq import allocate, Overlay

In [ ]:
resize_design = Overlay("resizer.xsa")

In [ ]:
dma = resize_design.axi_dma_0
resizer = resize_design.resize_accel_0

In [ ]:
image_path = "images/sahara.jpg"
original_image = Image.open(image_path)

In [ ]:
canvas = plt.gcf()
size = canvas.get_size_inches()
canvas.set_size_inches(size*2)

old_width, old_height = original_image.size
print("Image size: {}x{} pixels.".format(old_width, old_height))
plt.figure(figsize=(12, 10));
_ = plt.imshow(original_image)

In [ ]:
resize_factor = 2
new_width = int(old_width/resize_factor)
new_height = int(old_height/resize_factor)

In [ ]:
in_buffer = allocate(shape=(old_height, old_width, 3), 
                           dtype=np.uint8)
out_buffer = allocate(shape=(new_height, new_width, 3), 
                            dtype=np.uint8)

In [ ]:
in_buffer[:] = np.array(original_image)
print(in_buffer[-10:])

In [ ]:
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)

In [ ]:
resizer.write(0x00,0x81) # start

In [ ]:
dma.sendchannel.wait()
dma.recvchannel.wait()

In [ ]:
resized_image = Image.fromarray(out_buffer)

In [ ]:
del in_buffer
del out_buffer